# Marijuana Sales Prediction

## Projects Objective
- Goal: Predict sales of each brand
- Sub-problems: 

## Load Data
- Load from local dataset

In [393]:
%reload_ext autoreload
%autoreload 2
import warnings
warnings.filterwarnings('ignore')
# import required packages
import sys
import os
import pandas as pd
# Load Dataset
avgRetail = pd.read_csv("../data/BrandAverageRetailPrice.csv")
brandDetail = pd.read_csv("../data/BrandDetails.csv")
totalSales = pd.read_csv("../data/BrandTotalSales.csv")
totalUnits = pd.read_csv("../data/BrandTotalUnits.csv")

## Construct Dataset 
- Time-series data
    - Convert into same time-stamp format
- Clean Data 
    - without imputation
- Feature Engineering
    - Feature augmentation
    - Combining feature in same/different datasets

### Find the dataset to start with
- Criteria: Information of each datasets
- Objective: Since the final goal is to predict the future sales, we start with the dataset that seems to have strong correlation with sales, and also with few feature to start with. 

#### Total Sales dataset

In [394]:
totalSales.info()
totalSales.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25279 entries, 0 to 25278
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Months           25279 non-null  object
 1   Brand            25279 non-null  object
 2   Total Sales ($)  25279 non-null  object
dtypes: object(3)
memory usage: 592.6+ KB


,Months,Brand,Total Sales ($)
0,09/2018,10x Infused,"1,711.334232"
1,09/2018,1964 Supply Co.,"25,475.21594500000"
2,09/2018,3 Bros Grow,"120,153.644757"
3,09/2018,3 Leaf,"6,063.5297850000000"
4,09/2018,350 Fire,"631,510.0481550000"


#### Total units dataset

In [395]:
totalUnits.info()
totalUnits.head(5)
# find out how many brand do we have currently

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27686 entries, 0 to 27685
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Brands            27686 non-null  object 
 1   Months            27686 non-null  object 
 2   Total Units       25712 non-null  object 
 3   vs. Prior Period  24935 non-null  float64
dtypes: float64(1), object(3)
memory usage: 865.3+ KB


,Brands,Months,Total Units,vs. Prior Period
0,#BlackSeries,08/2020,"1,616.3390040000000",NaN
1,#BlackSeries,09/2020,NaN,-1.000000
2,#BlackSeries,01/2021,715.5328380000000,NaN
3,#BlackSeries,02/2021,766.669135,0.071466
4,#BlackSeries,03/2021,NaN,-1.000000


### Clean the choosen dataset
- Selected dataset: Total Sales
- Breakdown: Base-on different brand
- Reason: 
    - It contains more information about salses. 
    - It has much narrow data in brand features. 

#### Preprocess the selected dataset

In [396]:
# convert the time stamp dataset
totalSales["Months"] = totalSales["Months"].astype(str).astype('datetime64[ns]')
# convert salses data to numerical data form
totalSales["Total Sales ($)"] = totalSales["Total Sales ($)"].str.replace(',','')
totalSales["Total Sales ($)"] = totalSales["Total Sales ($)"].astype(float)
totalSales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25279 entries, 0 to 25278
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Months           25279 non-null  datetime64[ns]
 1   Brand            25279 non-null  object        
 2   Total Sales ($)  25279 non-null  float64       
dtypes: datetime64[ns](1), float64(1), object(1)
memory usage: 592.6+ KB


#### Find out all the brand name

In [397]:
brands = list(totalSales["Brand"].unique())
print("==================================")
print("Total: {} different brands".format(brands.__len__()))
print("==================================")

Total: 1627 different brands


#### Find the brand to start with
- Start with the brand that has much information. 

In [398]:
# start with one of the brand
# only list-out top three brand
totalSales["Brand"].value_counts().head(3)

Lift Ticket Laboratories    37
Garden Society              37
Field Extracts              37
Name: Brand, dtype: int64

### Feature Engineering 
- Transform and add more features in the selected dataset. 
- Add more features from other related datasets. 

#### Sales info about selected brand

In [399]:
brandName = 'Garden Society'
brandData = totalSales[totalSales.Brand == brandName].drop(['Brand'], 1)
brandData.head(10)

,Months,Total Sales ($)
162,2018-09-01,679.796207
632,2018-10-01,9847.971509
1126,2018-11-01,17585.544522
1659,2018-12-01,13796.748683
2191,2019-01-01,13525.256162
2727,2019-02-01,129564.314831
3288,2019-03-01,104925.011206
3858,2019-04-01,131054.937778
4446,2019-05-01,121876.366551
5047,2019-06-01,218701.840227


#### Adding Features to Dataset

In [400]:
#  adding from current dataset 
# add last month data
brandData.loc[:, 'Previous Month Sales'] = brandData.loc[:,"Total Sales ($)"].shift(-1)

#======================Adding Rolling Data======================
# add rolling sales data (for three months)
# calcualte the sum
rollSum, rollMonths = 0, 3
for month in range(1, rollMonths + 1): 
    rollSum += brandData.loc[:,"Total Sales ($)"].shift(-1 * month)
brandData.loc[:, 'Rolling Sales (3 months)'] = rollSum / rollMonths

# add rolling sales data (for six months)
rollSum, rollMonths = 0, 6
for month in range(1, rollMonths + 1): 
    rollSum += brandData.loc[:,"Total Sales ($)"].shift(-1 * month)
brandData.loc[:, 'Rolling Sales (6 months)'] = rollSum / rollMonths

brandData.head()

,Months,Total Sales ($),Previous Month Sales,Rolling Sales (3 months),Rolling Sales (6 months)
162,2018-09-01,679.796207,9847.971509,13743.421571,48207.474485
632,2018-10-01,9847.971509,17585.544522,14969.183122,68408.635530
1126,2018-11-01,17585.544522,13796.748683,52295.439892,85790.439202
1659,2018-12-01,13796.748683,13525.256162,82671.527400,119941.287792
2191,2019-01-01,13525.256162,129564.314831,121848.087938,151471.647204


##### Add from total_unit dataset

In [401]:
# general info
brandUnits = totalUnits[totalUnits.Brands == brandName]
brandUnits["Months"] = pd.to_datetime(brandUnits["Months"])
brandUnits["Total Units"] = brandUnits["Total Units"].str.replace(',','')
brandUnits["Total Units"] = brandUnits["Total Units"].astype(float)
brandUnits = brandUnits.drop(['Brands'], 1)
#===================Merge Data==================
brandData = brandData.merge(brandUnits, right_on="Months", left_on="Months")
brandData.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 37 entries, 0 to 36
Data columns (total 7 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   Months                    37 non-null     datetime64[ns]
 1   Total Sales ($)           37 non-null     float64       
 2   Previous Month Sales      36 non-null     float64       
 3   Rolling Sales (3 months)  34 non-null     float64       
 4   Rolling Sales (6 months)  31 non-null     float64       
 5   Total Units               37 non-null     float64       
 6   vs. Prior Period          37 non-null     float64       
dtypes: datetime64[ns](1), float64(6)
memory usage: 2.3 KB


##### Add from average retail price dataset

In [402]:
# Convert months column data into datetime
brandARP = avgRetail[avgRetail.Brands == brandName].drop(['Brands'], 1)
brandARP["Months"] = pd.to_datetime(brandARP["Months"])
# rename the features name
# since in the total_units dataset, ther is also a feature
# called "vs. Prior Period"
brandARP = brandARP.rename(columns={"vs. Prior Period" : "vs. Prior Period (ARP)"})

# merge the feature into the sales dataset)
brandData = brandData.merge(brandARP, left_on="Months", right_on="Months")
brandData.head(10)

,Months,Total Sales ($),Previous Month Sales,Rolling Sales (3 months),Rolling Sales (6 months),Total Units,vs. Prior Period,ARP,vs. Prior Period (ARP)
0,2018-09-01,679.796207,9847.971509,13743.421571,48207.474485,29.758195,-0.934166,22.844000,NaN
1,2018-10-01,9847.971509,17585.544522,14969.183122,68408.635530,390.194917,12.112184,25.238595,0.104824
2,2018-11-01,17585.544522,13796.748683,52295.439892,85790.439202,636.899760,0.632261,27.611165,0.094006
3,2018-12-01,13796.748683,13525.256162,82671.527400,119941.287792,506.255493,-0.205125,27.252541,-0.012988
4,2019-01-01,13525.256162,129564.314831,121848.087938,151471.647204,561.925335,0.109964,24.069490,-0.116798
5,2019-02-01,129564.314831,104925.011206,119285.438512,172060.027014,3045.884273,4.420443,42.537504,0.767279
6,2019-03-01,104925.011206,131054.937778,157211.048185,198743.628602,2593.505278,-0.148521,40.456833,-0.048914
7,2019-04-01,131054.937778,121876.366551,181095.206469,197857.786242,3234.843260,0.247286,40.513536,0.001402
8,2019-05-01,121876.366551,218701.840227,224834.615515,200401.076717,3673.502005,0.135604,33.177161,-0.181085
9,2019-06-01,218701.840227,202707.412629,240276.209019,186254.110968,6021.450299,0.639158,36.320459,0.094743


##### Add from brand details dataset

In [407]:
# info aobut brain details
brandDetail[]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 144977 entries, 0 to 144976
Data columns (total 25 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   State                144977 non-null  object 
 1   Channel              144977 non-null  object 
 2   Category L1          144977 non-null  object 
 3   Category L2          144977 non-null  object 
 4   Category L3          144245 non-null  object 
 5   Category L4          102618 non-null  object 
 6   Category L5          50135 non-null   object 
 7   Brand                144977 non-null  object 
 8   Product Description  144977 non-null  object 
 9   Total Sales ($)      144977 non-null  object 
 10  Total Units          144977 non-null  object 
 11  ARP                  144977 non-null  float64
 12  Flavor               7807 non-null    object 
 13  Items Per Pack       144977 non-null  int64  
 14  Item Weight          64454 non-null   object 
 15  Total THC        

## Explore Data 
- Visualize Data 
- Explore data (correlation)
- Objective of features combination

In [404]:
import matplotlib.pyplot as plt
import seaborn as sns

## Data Preprocessing
- Develope several pipeline
- Visualize pipelines 
- (pipeline update -- base on training result) 

In [405]:
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline



## Hyper-parameters
- Test/Train ratio
- Hyperparameters for traning
- Hyperparameters for model 

## Split Datasets
- Split into training/validation and testing dataset

In [406]:
from sklearn.model_selection import train_test_split

## Deploy ML/DL
- Model selection
- Comparison between each baseline models
- Select best baseline model
- Search for best parameters (grid/random search)

## Visualize Result 
- Visualization result
    - Tableau
    - Seaborn
- Check important features 
    - Go back to exploring data (if necessary)